# Решение с помощью классификатора kNN

### Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
id_age_train = pd.read_csv('age_profile_train', sep = '\t', header = None)
id_age_train.columns = ['Id', 'age']
id_age_train.head()

,Id,age
0,000000013CB5719C0000A2C90002C101,53
1,00000001442BE24000001B7D00F50801,48
2,00000001448580F800003F1B31FB0901,28
3,0000000145BDB2FF000157971645E901,44
4,000000014602771F0000DB9359714C01,48


In [3]:
age_by_id = dict(id_age_train.values)
print age_by_id[id_age_train['Id'].values[0]]

53


In [4]:
train_urls = pd.read_csv('url_domain_train', sep = '\t', header = None)
train_urls.columns = ['Id', 'url', 'cnt']
train_urls.head()

,Id,url,cnt
0,000000014B60815F65B38258011B6C01,login.rutracker.org,1
1,000000014B60815F65B38258011B6C01,rutracker.org,4
2,000000014C03DA2A47AC433A0C755201,admin.tour-spb.net,1
3,000000014C03DA2A47AC433A0C755201,czinfo.ru,1
4,000000014C03DA2A47AC433A0C755201,forumsostav.ru,1


In [5]:
test_urls = pd.read_csv('url_domain_test', sep = '\t', header = None)
test_urls.columns = ['Id', 'url', 'cnt']
test_urls.head()

,Id,url,cnt
0,0000000151004FF4ADD746DA10685A01,afisha.ru,2
1,0000000151004FF4ADD746DA10685A01,aif.ru,1
2,0000000151004FF4ADD746DA10685A01,aimfar.solution.weborama.fr,1
3,0000000151004FF4ADD746DA10685A01,alkotest.ru,1
4,0000000151004FF4ADD746DA10685A01,aptekamos.ru,1


### Обработка данных

Объединим train и test в один датафрейм

In [6]:
train_test_data = pd.DataFrame(np.vstack([train_urls.values, test_urls.values]))
train_test_data.columns = ['Id', 'url', 'cnt']
train_test_data.head()

,Id,url,cnt
0,000000014B60815F65B38258011B6C01,login.rutracker.org,1
1,000000014B60815F65B38258011B6C01,rutracker.org,4
2,000000014C03DA2A47AC433A0C755201,admin.tour-spb.net,1
3,000000014C03DA2A47AC433A0C755201,czinfo.ru,1
4,000000014C03DA2A47AC433A0C755201,forumsostav.ru,1


Оставим лишь 10000 сайтов, построим на них sparse matrix

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features = 10000)

In [8]:
groupbyed = train_test_data.groupby('Id')['url'].apply(lambda x: x.tolist())
print type(groupbyed), groupbyed.shape

<class 'pandas.core.series.Series'> (138577,)


In [9]:
groupbyed1 = zip(groupbyed.keys(), groupbyed.values)
print len(groupbyed1), len(groupbyed1[0])

138577 2


In [10]:
groupbyed1.sort(key = lambda x: x[0] in age_by_id, reverse = True)
for i in range(len(groupbyed1)):
    if groupbyed1[i][0] not in age_by_id:
        print i
        real_train_size = i
        break

118603


In [11]:
vectorized_data = vectorizer.fit_transform([''.join(x[1]) for x in groupbyed1])
print vectorized_data.shape

(138577, 10000)


### Обучение модели

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [13]:
ages = [age_by_id[groupbyed1[i][0]] for i in range(real_train_size)]
knn.fit(vectorized_data[:real_train_size], ages)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [14]:
%%time
from sklearn.cross_validation import cross_val_score
print cross_val_score(knn, vectorized_data[:real_train_size], ages, scoring = 'mean_squared_error')

MemoryError: 